<a href="https://colab.research.google.com/github/shekhawatmeenu18/NLP_Projects/blob/main/Spam_detection_ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [1]:
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup  ## For cleaning the data
import re

import torch
from transformers import AutoModel, AutoTokenizer, AlbertForSequenceClassification, Trainer, TrainingArguments,EarlyStoppingCallback
from datasets import Dataset

In [2]:
import numpy as np
import pandas as pd
import os
import json

In [9]:
# # Upload the kaggle.json file
# from google.colab import files
# files.upload()

# # Move the kaggle.json file to the .kaggle directory
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# # Load the API key and username from the file
# with open('/root/.kaggle/kaggle.json', 'r') as f:
#     kaggle_data = json.load(f)

# os.environ["KAGGLE_KEY"] = kaggle_data["key"]
# os.environ["KAGGLE_USERNAME"] = kaggle_data["username"]

# !kaggle datasets list

# !kaggle datasets download -d /kaggle/input/email-spam-classification-dataset/combined_data.csv

In [5]:
# for dirname, _, filenames in os.walk('/content'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

/content/combined_data.csv.zip
/content/.config/active_config
/content/.config/config_sentinel
/content/.config/gce
/content/.config/default_configs.db
/content/.config/.last_update_check.json
/content/.config/.last_survey_prompt.yaml
/content/.config/.last_opt_in_prompt.yaml
/content/.config/configurations/config_default
/content/.config/logs/2024.06.12/18.21.44.115760.log
/content/.config/logs/2024.06.12/18.21.56.470820.log
/content/.config/logs/2024.06.12/18.21.35.197821.log
/content/.config/logs/2024.06.12/18.21.55.790625.log
/content/.config/logs/2024.06.12/18.20.59.291508.log
/content/.config/logs/2024.06.12/18.21.24.034464.log
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/mnist_test.csv
/content/sample_data/mnist_train_small.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/california_housing_test.csv


In [3]:
path = '/content/combined_data.csv'
data = pd.read_csv(path)
data.shape

(83448, 3)

In [7]:
#  data.to_csv("combined_data.csv")

In [4]:
data.sample(5).T

,480,81364,53926,68138,11869
Unnamed: 0,480,81364,53926,68138,11869
label,1,0,0,1,1
text,+ unable to see graphics ? please go here to v...,track manage your source code and digital ass...,the drn number assigned is : 291037 . sorry i ...,"your membership exchange , issue # 442\naugust...",working on your company ' s image ? start with...


In [5]:
data.drop(columns=['Unnamed: 0'], inplace=True)
data.shape

(83448, 2)

In [6]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").get_text()
    text = re.sub(r"https?://\S+|www\.\S+"," ",text) # Removing URLS
    text = re.sub(r"@[A-Za-z0-9]+"," ",text)
    text = re.sub(r"https?://[A-Za-z0-9]+"," ",text) # Removing html tags
    text = re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});"," ",text) # Removing html tags
    text = re.sub(r"[^\w\s]", " ", text) # Removing Punctuation
    text = re.sub(r" +", " ", text) # Removing repeatition of anything
#     text = re.sub(r"[^a-zA-Z.!?']", " ", text) # Removing non-letter words
    text = re.sub(r"\s+", " ", text).strip() # Removing whitespace
    return text

In [7]:
data['text'] = data['text'].apply(clean_text)
data.sample(5).T

,28000,12358,46461,71456,70779
label,1,0,1,1,0
text,we only sell premium watches there s no batter...,metze on tue may escapenumber escapenumber at ...,grandes lanzamientos mayo escapenumber son sup...,with get smart skills the efforts oftenwould w...,on escapenumber escapenumber escapenumber ravi...


In [8]:
# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(data)

In [9]:
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 83448
})

In [10]:
# Load the ALBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

In [11]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

In [12]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/83448 [00:00<?, ? examples/s]

In [27]:
# Split the dataset into train and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.6)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

train_eval_split = train_dataset.train_test_split(test_size=0.4)
train_dataset = train_eval_split['train']
eval_dataset = train_eval_split['test']

In [30]:
train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20027
})

In [31]:
# Load the ALBERT model
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
# ## for Hyper-parameter tuning run OPTUNA
# !pip install optuna

# def objective(trial):
#     # Define hyperparameters to search
#     training_args = TrainingArguments(
#         output_dir='./results',
#         evaluation_strategy="epoch",
#         per_device_train_batch_size=trial.suggest_categorical('per_device_train_batch_size', [8, 16, 32]),
#         per_device_eval_batch_size=trial.suggest_categorical('per_device_eval_batch_size', [8, 16, 32]),
#         num_train_epochs=trial.suggest_int('num_train_epochs', 2, 5),
#         learning_rate=trial.suggest_loguniform('learning_rate', 1e-5, 5e-5),
#         weight_decay=trial.suggest_loguniform('weight_decay', 0.01, 0.3),
#         logging_dir='./logs',
#         load_best_model_at_end=True,
#         metric_for_best_model='accuracy'
#     )

#     trainer = Trainer(
#         model_init=model_init,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=test_dataset,
#         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
#     )

#     # Train the model
#     trainer.train()

#     # Evaluate the model
#     eval_result = trainer.evaluate()

#     # The objective is to minimize the loss, return the appropriate metric
#     return eval_result['eval_loss']


# # Use Optuna to search for the best hyperparameters
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=20)

# # Print the best hyperparameters found
# print("Best hyperparameters: ", study.best_params)

# # Train the final model with the best hyperparameters
# best_params = study.best_params
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy="epoch",
#     per_device_train_batch_size=best_params['per_device_train_batch_size'],
#     per_device_eval_batch_size=best_params['per_device_eval_batch_size'],
#     num_train_epochs=best_params['num_train_epochs'],
#     learning_rate=best_params['learning_rate'],
#     weight_decay=best_params['weight_decay'],
#     logging_dir='./logs',
#     load_best_model_at_end=True,
#     metric_for_best_model='accuracy'
# )


In [39]:
## If training_args code won't work, just install below libraries and restart the kernel
# !pip install transformers[torch]
# !pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [32]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [33]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")
trainer.save_model()
# trainer.push_to_hub()

In [ ]:
# Get predictions on the validation set
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
labels = predictions.label_ids

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score

# Compute metrics
accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds, average='binary')
recall = recall_score(labels, preds, average='binary')
f1 = f1_score(labels, preds, average='binary')
roc_auc = roc_auc_score(labels, preds)
conf_matrix = confusion_matrix(labels, preds)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"Confusion Matrix:\n {conf_matrix}")

In [ ]:
# Example texts for prediction
texts = ["Call me on this message for winning a $100 price money.", "You phone recharge is due. Please recharge soon."]

# Tokenize the texts
tokenized_texts = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(**tokenized_texts)

# Convert logits to probabilities
probs = torch.nn.functional.softmax(outputs.logits, dim=1)

# Get predicted class
predicted_class = probs.argmax(dim=1)
print(f"Predicted class: {predicted_class}")

In [1]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [2]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
!git clone https://github.com/shekhawatmeenu18/NLP_Projects.git

Cloning into 'NLP_Projects'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 39.22 KiB | 2.61 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [5]:
%cd NLP_Projects

/content/NLP_Projects


In [6]:
# Configure Git with your name and email
!git config --global user.name "shekhawatmeenu18"
!git config --global user.email "shekhawatmeenu18@gmail.com"

In [7]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
